In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys
import time
from datetime import datetime, timedelta
import talib
import tushare as ts
import datetime

In [2]:
start = time.time()

In [3]:
#显示所有列
pd.set_option("display.max_columns", None) 
#显示所有行，慎用，容易死机
#pd.set_option("display.max_rows", None) 
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

# 1、读入数仓数据

In [4]:
# 读取year
def read_newyear():
    df = pd.read_csv('../JN_DataWarehouse/stock_analysis/TuShare/new_year.csv' ,index_col=False)
    
    del df['Unnamed: 0']
    return df

In [5]:
df_stock_all = read_newyear()
df_stock_all.head()

FileNotFoundError: [Errno 2] File ../JN_DataWarehouse/stock_analysis/TuShare/new_year.csv does not exist: '../JN_DataWarehouse/stock_analysis/TuShare/new_year.csv'

# 2、 RSI运算

In [ ]:
stock_code = '002508'
today = datetime.date.today()                #datetime.date类型日期  today = datetime.date(2019,11,15)
today = today -  datetime.timedelta(365*0)
A3yday = today - datetime.timedelta(365*0.5)
startDate = A3yday.strftime('%Y%m%d')        #字符串型日期
endDate = today.strftime('%Y%m%d')
from Stock_Interface import read_finance_year   
stock_name =  read_finance_year.read_companyInfo(stock_code)

df = df_stock_all.query("ts_code.str.startswith('"+stock_code+"')",engine='python')    
df = df.query("trade_date > "+startDate+" & trade_date < "+endDate+"")

# 日期转换
df["trade_date"] = pd.to_datetime(df["trade_date"], format='%Y%m%d')
df["trade_date"] = df['trade_date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [ ]:
df

In [ ]:
if df.empty ==True:
    print(" df is empty ")
    sys.exit(2)
 
df = df[ df['trade_date'] > '2020-01-01']
if len(df) <10:
    print(" len(df) <10 ")
    sys.exit(2)
 
df['ma10'] = df['close'].rolling(window=10).mean()
df.index = pd.to_datetime(df.trade_date)
dw = pd.DataFrame()
#  baike.baidu.com/item/rsi顺势指标
dw['rsi6'] = talib.RSI(df.close, timeperiod=6)
dw['rsi12'] = talib.RSI(df.close, timeperiod=12)
dw['rsi24'] = talib.RSI(df.close, timeperiod=24)
print("rsi6={0:.1f} , rsi12={1:.1f}, rsi24={1:.1f}".format(dw['rsi6'][-1], dw['rsi12'][-1], dw['rsi24'][-1]))
 
# 画股票收盘价图
fig,axes = plt.subplots(2,1)

df[['close', 'ma10']].plot(ax=axes[0], grid=True, title=stock_code+stock_name, figsize=(16, 7))
# 画 RSI 曲线图
dw[['rsi6', 'rsi12', 'rsi24']].plot(ax=axes[1], grid=True)
plt.legend(loc='best', shadow=True)

plt.show()

In [ ]:
dw.tail(10)